# DeepDream

We'll describe an implementation of DeepDream, a system that uses neural networks to generate dream-like images, showcasing how algorithms can reinterpret and morph visual content in surreal and artistic ways.

Note tha several of these steps can take a few minutes to complete, especially on computers without GPU.

## Select Imput Image

We begin by loading the example image titled 'neuraltissue_with_colorlabels.png'. 

This image is sourced from the Drosophila ssTEM dataset, which is publicly available on Figshare: [Segmented anisotropic ssTEM dataset of neural tissue](https://figshare.com/articles/dataset/Segmented_anisotropic_ssTEM_dataset_of_neural_tissue/856713). This dataset provides a detailed view of neural tissue, aiding in the study of neural structures and patterns. The image can also be downloaded from the corresponding GitHub repository at [this link](http://github.com/unidesigner/groundtruth-drosophila-vnc), which offers additional resources and information related to the Drosophila ssTEM dataset.

In [ ]:
from PIL import Image

im = Image.open("neuraltissue_with_colorlabels.png").convert('RGB').resize((256, 256))

The image is a part of the **Drosophila ssTEM dataset** and features a cross-section of neural tissue from a Drosophila melanogaster third instar larva ventral nerve cord. This high-resolution dataset is crucial for studying the details of neural structures and for the development of image segmentation algorithms.

Each image in the dataset, including the one provided, comes with a detailed segmentation of the neural tissue. In the images, various neural structures are labeled with different colors to facilitate identification:
- **Red**: Neuron cell bodies
- **Purple**: Glia or extracellular matrix
- **White lines**: Neuron membranes
- **Other shades**: Mitochondria and synapses (not explicitly colored in the provided image)

The dataset provides the following specifications:
- **Stack Size**: Approximately 4.7 x 4.7 x 1 microns
- **Resolution**: 4.6 x 4.6 nm/pixel
- **Section Thickness**: Between 45-50 nm

For more detailed information and access to the dataset, refer to the [Figshare repository](https://figshare.com/articles/dataset/Segmented_anisotropic_ssTEM_dataset_of_neural_tissue/856713) and the [GitHub repository](https://github.com/unidesigner/groundtruth-drosophila-vnc).

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(im)
plt.axis('off')
plt.show()

## Load Pretrained Neural Network

We import the VGG16 model, a pretrained neural network known for its proficiency in image recognition tasks, with weights initialized from the ImageNet dataset. We then set the model to evaluation mode and freeze all weights to prevent further changes during our operations.

In [ ]:
import torchvision.models as models
from torchvision.models import VGG16_Weights

model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

model.eval()
model.requires_grad_(False)

print(model)

## Implement DeepDream

The DeepDream algorithm is designed to optimize an image's representation by performing loss maximization using gradient ascent. By iterating over the image data and adjusting it in the direction that increases the activation of certain layers within a pretrained neural network, this function effectively "dreams up" new visual patterns and textures that amplify the features those layers detect. The process involves normalizing the image data (VGG16 is pretrained on the ImageNet dataset and the inputs are normalized wih respect to the mean and standard deviation of the channels of this dataset), applying forward hooks to capture layer activations, calculating the loss, and then updating the image based on the gradients obtained. The result is an altered image that highlights the intricate patterns learned by the neural network.

We add to `fnc_deepdream.py` two functions to transform images to tensors, and vice versa.

The `image_to_tensor()` function preprocess the image by converting to tensor and normalizing the image.
```python
def image_to_tensor(im, mean, std):
    import torchvision.transforms as tt

    normalize = tt.Compose([tt.ToTensor(), tt.Normalize(mean, std)])

    return normalize(im).unsqueeze(0).requires_grad_(True)
```

The `tensor_to_image()` function postprocesses the tensor converting it back to the image format.

```python
def tensor_to_image(image, mean, std):
    import torchvision.transforms as tt
    import numpy as np
    from PIL import Image

    denormalize = tt.Normalize(mean=-mean / std, std=1 / std)

    im_array = denormalize(image.data.clone().detach().squeeze()).numpy()
    im_array = np.clip(im_array.transpose(1, 2, 0) * 255, 0, 255).astype(np.uint8)
    return Image.fromarray(im_array, 'RGB')
```

In [ ]:
layer = model.features[1]
iter_num=100
step=.1

import numpy as np
import torch
from fnc_deepdream import image_to_tensor, tensor_to_image

# Normalization parameters typically used with pretrained models
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

# Define bounds for normalized image values
low = torch.tensor((-mean / std).reshape(1, -1, 1, 1))
high = torch.tensor(((1 - mean) / std).reshape(1, -1, 1, 1))

# Image to tensor
im_tensor = image_to_tensor(im, mean, std)

# Perform DeepDream iterations
hookdata = {}

def hook_func(layer, input, output):
    hookdata['activations'] = output

for _ in range(iter_num):
    handle = layer.register_forward_hook(hook_func)
    try:
        _ = model(im_tensor) # No output is needed, as we just need the hooks obtained from the forward pass
    except Exception as e:
        print(f"An error occurred during model predition: {e}")
    finally:
        handle.remove()

    # Calculate mean activation for each layer and sum them as total loss
    loss = hookdata['activations'].mean()
    loss.backward()

    # Calculate the normalized gradient
    grad_mean = torch.mean(im_tensor.grad.data)
    grad_std = torch.std(im_tensor.grad.data)
    normalized_grad = (im_tensor.grad.data - grad_mean) / (grad_std + 1e-8)

    # Perform the gradient ascent step
    im_tensor.data += step * normalized_grad
    
    # Clear gradients for next iteration
    im_tensor.grad.zero_()

    # Clamp the image data to ensure pixel values are valid
    im_tensor.data.clamp_(low, high)

# Tensor to Image
im_deepdream = tensor_to_image(im_tensor, mean, std)

# Plot
plt.imshow(im_deepdream)
plt.title("DeepDream for Layer 1")
plt.axis("off")
plt.show()


### Refactor Code as a Function

We now refactor the previous code as a function that takes as input an image and a layer number, and calculates, returns, and plot the DeepDream.

In [ ]:
def deepdream(im, layer_index, iter_num=100, step=.1):
    import numpy as np
    import torch
    import matplotlib.pyplot as plt
    from fnc_deepdream import image_to_tensor, tensor_to_image

    # Normalization parameters typically used with pretrained models
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

    # Define bounds for normalized image values
    low = torch.tensor((-mean / std).reshape(1, -1, 1, 1))
    high = torch.tensor(((1 - mean) / std).reshape(1, -1, 1, 1))

    # Image to tensor
    im_tensor = image_to_tensor(im, mean, std)

    # Perform DeepDream iterations
    hookdata = {}

    def hook_func(layer, input, output):
        hookdata['activations'] = output

    layer = model.features[layer_index]
    for _ in range(iter_num):
        handle = layer.register_forward_hook(hook_func)
        try:
            _ = model(im_tensor) # No output is needed, as we just need the hooks obtained from the forward pass
        except Exception as e:
            print(f"An error occurred during model predition: {e}")
        finally:
            handle.remove()

        # Calculate mean activation for each layer and sum them as total loss
        loss = hookdata['activations'].mean()
        loss.backward()

        # Calculate the normalized gradient
        grad_mean = torch.mean(im_tensor.grad.data)
        grad_std = torch.std(im_tensor.grad.data)
        normalized_grad = (im_tensor.grad.data - grad_mean) / (grad_std + 1e-8)

        # Perform the gradient ascent step
        im_tensor.data += step * normalized_grad
        
        # Clear gradients for next iteration
        im_tensor.grad.zero_()

        # Clamp the image data to ensure pixel values are valid
        im_tensor.data.clamp_(low, high)

    # Tensor to Image
    im_deepdream = tensor_to_image(im_tensor, mean, std)

    # Plot
    plt.imshow(im_deepdream)
    plt.title(f"DeepDream at Layer {layer_index}")
    plt.axis("off")
    plt.show()

In [ ]:
deepdream(im, layer_index=1, iter_num=100, step=.1)

### Refactor Code with Context Manager

We now refactor the previous function with a context manager instead of a try-exempt construct. For this, we need the `fwd_hook` class.

In [ ]:
class fwd_hook():
    def __init__(self, layer):
        self.hook = layer.register_forward_hook(self.hook_func)

    def hook_func(self, layer, input, output):
        self.activations = output

    def __enter__(self, *args): 
        return self
    
    def __exit__(self, *args): 
        self.hook.remove()

In [ ]:
def deepdream(im, layer_index, iter_num=100, step=.1):
    import numpy as np
    import torch
    import matplotlib.pyplot as plt
    from fnc_deepdream import image_to_tensor, tensor_to_image

    # Normalization parameters typically used with pretrained models
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

    # Define bounds for normalized image values
    low = torch.tensor((-mean / std).reshape(1, -1, 1, 1))
    high = torch.tensor(((1 - mean) / std).reshape(1, -1, 1, 1))

    # Image to tensor
    im_tensor = image_to_tensor(im, mean, std)

    # Perform DeepDream iterations
    layer = model.features[layer_index]
    for _ in range(iter_num):
        with fwd_hook(layer) as fh:
            _ = model(im_tensor) # No output is needed, as we just need the hooks obtained from the forward pass

        # Calculate mean activation for each layer and sum them as total loss
        loss = fh.activations.mean()
        loss.backward()

        # Calculate the normalized gradient
        grad_mean = torch.mean(im_tensor.grad.data)
        grad_std = torch.std(im_tensor.grad.data)
        normalized_grad = (im_tensor.grad.data - grad_mean) / (grad_std + 1e-8)

        # Perform the gradient ascent step
        im_tensor.data += step * normalized_grad
        
        # Clear gradients for next iteration
        im_tensor.grad.zero_()

        # Clamp the image data to ensure pixel values are valid
        im_tensor.data.clamp_(low, high)

    # Tensor to Image
    im_deepdream = tensor_to_image(im_tensor, mean, std)

    # Plot
    plt.imshow(im_deepdream)
    plt.title(f"DeepDream at Layer {layer_index}")
    plt.axis("off")
    plt.show()

In [ ]:
deepdream(im, layer_index=1, iter_num=100, step=.1)

### DeepDream Deeper Layers

We can now apply DeepDreams to deeper layers.

In [ ]:
for layer_index in [1, 3, 6, 8, 11, 13, 15, 18, 20, 22, 25, 27, 29]:
    deepdream(im, layer_index, iter_num=100, step=.1)

## Combine Multiple Layers in a Single DeepDream

For doing this, we need to upgrade the context management class to containt the forward hooks for multiple layers, which we now add to `fc_deepdream.py`:

```python
class fwd_hooks():
    def __init__(self, layers):
        self.hooks = []
        self.activations = []
        for layer in layers:
            self.hooks.append(layer.register_forward_hook(self.hook_func))

    def hook_func(self, layer, input, output):
        self.activations.append(output)

    def __enter__(self, *args): 
        return self
    
    def __exit__(self, *args): 
        for hook in self.hooks:
            hook.remove()
```

We then upgrade the `deepdream()` function to accept multiple layers as input.

In [ ]:
def deepdream(im, layer_indices, iter_num=100, step=.1):
    import numpy as np
    import torch
    import matplotlib.pyplot as plt
    from fnc_deepdream import fwd_hooks, image_to_tensor, tensor_to_image

    # Normalization parameters typically used with pretrained models
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

    # Define bounds for normalized image values
    low = torch.tensor((-mean / std).reshape(1, -1, 1, 1))
    high = torch.tensor(((1 - mean) / std).reshape(1, -1, 1, 1))

    # Image to tensor
    im_tensor = image_to_tensor(im, mean, std)

    # Perform DeepDream iterations
    layers = [model.features[i] for i in layer_indices]
    for _ in range(iter_num):
        with fwd_hooks(layers) as fh:
            _ = model(im_tensor) # No output is needed, as we just need the hooks obtained from the forward pass

        # Calculate mean activation for each layer and sum them as total loss
        losses = [activations.mean() for activations in fh.activations_list]
        loss = torch.stack(losses).sum()
        loss.backward()

        # Calculate the normalized gradient
        grad_mean = torch.mean(im_tensor.grad.data)
        grad_std = torch.std(im_tensor.grad.data)
        normalized_grad = (im_tensor.grad.data - grad_mean) / (grad_std + 1e-8)

        # Perform the gradient ascent step
        im_tensor.data += step * normalized_grad
        
        # Clear gradients for next iteration
        im_tensor.grad.zero_()

        # Clamp the image data to ensure pixel values are valid
        im_tensor.data.clamp_(low, high)

    # Tensor to Image
    im_deepdream = tensor_to_image(im_tensor, mean, std)

    # Plot
    plt.imshow(im_deepdream)
    plt.title(f"DeepDream at Layers {layer_indices}")
    plt.axis("off")
    plt.show()

    return im_deepdream

We now calculate a DeepDream combining multiple layers.

In [ ]:
deepdream(im, layer_indices=[1, 8, 11, 18, 25, 27, 29], iter_num=100, step=.1);

## Combine Different Resolutions using Octaves

We can also combine images at different resolutions using octaves, a technique often used in the DeepDream algorithm, which involves processing the image at multiple scales. This technique is used to enhance the effects of the DeepDream algorithm by capturing and emphasizing patterns at different levels of granularity. 

In [ ]:
octave_scale = 1.4
layers_indices = [18]

original_size = im.size

# Iterate over the range of octaves
im_deepdream = im
for octave in range(-2, 3):
    # Resize the image for the current octave
    new_size = (
        int(original_size[0] * (octave_scale ** octave)), 
        int(original_size[1] * (octave_scale ** octave)),
    )
    im_deepdream = im_deepdream.resize(new_size, Image.LANCZOS)

    # Apply DeepDream to the resized image
    im_deepdream = deepdream(im_deepdream, layers_indices, 
                             iter_num=100, step=.1)

    # Resize the processed image back to the original size
    im_deepdream = im_deepdream.resize(original_size, Image.LANCZOS)

    plt.imshow(im_deepdream)
    plt.title(f"Octave {octave}")
    plt.axis('off')
    plt.show()

We can furthermore use the output of several layers simultaneously to enhance multiple features.

In [ ]:
octave_scale = 1.4
layer_indices = [15, 18, 20, 22, 25, 27, 29]

original_size = im.size

# Iterate over the range of octaves
im_deepdream = im
for octave in range(-2, 3):
    # Resize the image for the current octave
    new_size = (
        int(original_size[0] * (octave_scale ** octave)), 
        int(original_size[1] * (octave_scale ** octave)),
    )
    im_deepdream = im_deepdream.resize(new_size, Image.LANCZOS)

    # Apply DeepDream to the resized image
    im_deepdream = deepdream(im_deepdream, layer_indices, 
                             iter_num=100, step=.1)

    # Resize the processed image back to the original size
    im_deepdream = im_deepdream.resize(original_size, Image.LANCZOS)

    plt.imshow(im_deepdream)
    plt.title(f"Octave {octave}")
    plt.axis("off")
    plt.show()
